<a href="https://colab.research.google.com/github/matheus896/MyGit/blob/main/Agents_Research%2C_Writer_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --q crewai litellm

In [ ]:
from crewai import Agent, Task, Crew, LLM
from litellm import completion
import os

# Warning control
import warnings
warnings.filterwarnings('ignore')


os.environ["SAMBANOVA_API_KEY"] = "API_KEY"
LLM = LLM ("sambanova/Meta-Llama-3.1-405B-Instruct",
           temperature=0.2)

# Criação de agentes
# Defina seus agentes e dê a eles role, goal e backstory

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=LLM
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=LLM
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=LLM
)

# Criação de tarefas
# Defina suas tarefas e forneça a elas uma descrição, expected_output e agente.

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
        "The text should be in Portuguese BR",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
        "The text should be in Portuguese BR",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs."
                    "The text should be in Portuguese BR",
    agent=editor
)

# Criação da crew
# Crie sua equipe de agentes
# Passe as tarefas a serem executadas por esses agentes.
# Observação: neste exemplo simples, as tarefas serão executadas sequencialmente (ou seja, elas dependem umas das outras), portanto, a ordem da tarefa na lista é importante.
# verbose=True permite que você veja todos os registros da execução.

crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

# Executando a equipe
# Observação: os LLMs podem fornecer saídas diferentes para a mesma entrada

result = crew.kickoff(inputs={"topic": "Generative Artificial Intelligence"})

# Exiba os resultados de sua execução como markdown no notebook.

# Exibir o resultado final como Markdown.

# from IPython.display import Markdown
# Markdown(result)